##NOTE: 
For models trained before ~Feb 2021, this requires an older version of Pykeen to be used. 
Use version pykeen==1.1.0

In [1]:

%load_ext autoreload
%autoreload 2
import os
import sys
import time
import pickle
from tqdm import tqdm
# for some reason, need to go to the sheaf_kg directory in order for torch.load to work
os.chdir('/home/gebhart/projects/sheaf_kg/sheaf_kg')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pykeen
import torch
from pykeen.pipeline import pipeline
from pykeen.models import TransE

In [2]:
dataset = 'FB15k-237'
num_test = 200
train_test_queries = 'train'
model_name = 'TransE_1000epochs_128embdim_CrossEntropyLossloss_42seed_20210128-1828'
save_loc = '/home/gebhart/projects/sheaf_kg/data/{}/{}/trained_model.pkl'.format(dataset,model_name)
betae_path = '/home/gebhart/projects/sheaf_kg/data/{}-betae'.format(dataset)
model = torch.load(save_loc).to('cpu')

In [3]:
query_structures = [('e', ('r', 'r')), ('e', ('r', 'r', 'r')), (('e', ('r',)), ('e', ('r',))), (('e', ('r',)), ('e', ('r',)), ('e', ('r',)))]

query_name_dict = {('e',('r',)): '1p', 
                    ('e', ('r', 'r')): '2p',
                    ('e', ('r', 'r', 'r')): '3p',
                    (('e', ('r',)), ('e', ('r',))): '2i',
                    (('e', ('r',)), ('e', ('r',)), ('e', ('r',))): '3i',
                    ((('e', ('r',)), ('e', ('r',))), ('r',)): 'ip',
                    (('e', ('r', 'r')), ('e', ('r',))): 'pi',
                    (('e', ('r',)), ('e', ('r', 'n'))): '2in',
                    (('e', ('r',)), ('e', ('r',)), ('e', ('r', 'n'))): '3in',
                    ((('e', ('r',)), ('e', ('r', 'n'))), ('r',)): 'inp',
                    (('e', ('r', 'r')), ('e', ('r', 'n'))): 'pin',
                    (('e', ('r', 'r', 'n')), ('e', ('r',))): 'pni',
                    (('e', ('r',)), ('e', ('r',)), ('u',)): '2u-DNF',
                    ((('e', ('r',)), ('e', ('r',)), ('u',)), ('r',)): 'up-DNF',
                    ((('e', ('r', 'n')), ('e', ('r', 'n'))), ('n',)): '2u-DM',
                    ((('e', ('r', 'n')), ('e', ('r', 'n'))), ('n', 'r')): 'up-DM'
                }

In [4]:
ds = pykeen.datasets.get_dataset(dataset=dataset)
training = ds.training.mapped_triples
relid2label = ds.training.relation_id_to_label 
label2relid = {v:k for k,v in relid2label.items()}

entid2label = ds.training.entity_id_to_label 
label2entid = {v:k for k,v in entid2label.items()}

You're trying to map triples with 30 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 28 from 20466 triples were filtered out


In [5]:
with open(os.path.join(betae_path,f'{train_test_queries}-queries.pkl'), 'rb') as f:
    test_queries = pickle.load(f)

if train_test_queries == 'test':
    with open(os.path.join(betae_path,f'{train_test_queries}-easy-answers.pkl'), 'rb') as f:
        test_answers = pickle.load(f)
elif train_test_queries == 'train':
    with open(os.path.join(betae_path,f'{train_test_queries}-answers.pkl'), 'rb') as f:
        test_answers = pickle.load(f)
    
with open(os.path.join(betae_path,'id2rel.pkl'), 'rb') as f:
    id2rel = pickle.load(f)
    
with open(os.path.join(betae_path,'id2ent.pkl'), 'rb') as f:
    id2ent = pickle.load(f)

In [6]:
def map_ent(e):
    return label2entid[id2ent[e]]
def map_rel(r):
    orientation = 1
    relname = id2rel[r]
    if relname[0] == '-':
        orientation = -1
    return label2relid[relname[1:]], orientation

In [7]:
def L_p(query, model):
    '''query of form ('e', ('r', 'r', ... , 'r')).
    here we assume 2 or more relations are present so 2p or greater
    '''
    ent = map_ent(query[0])
    invs = []
    rels = []
    for r in query[1]:
        mapped_id, orientation = map_rel(r) 
        rels.append(mapped_id)
        invs.append(orientation)
    n_path_ents = len(rels)
    
    edge_indices = np.concatenate([np.arange(0,n_path_ents)[:,np.newaxis].T, np.arange(1,n_path_ents+1)[:,np.newaxis].T], axis=0)
    
    rel_idx_tensor = torch.LongTensor(rels)
    relations = model.relation_embeddings(indices=rel_idx_tensor)
    
    for invix in range(len(invs)):
        if invix == -1:
            relations[invix] = -relations[invix]
    
    ent_idx_tensor = torch.LongTensor([ent])
    source_embeddings = model.entity_embeddings(indices=ent_idx_tensor)
    sm = source_embeddings + torch.sum(relations, dim=0)
    return sm

def L_i(query, model):
    '''query of form (('e', ('r',)), ('e', ('r',)), ... , ('e', ('r',)))'''
    num_intersects = len(query)
    ents = []
    rels = []
    invs = []
    for pair in query:
        ents.append(map_ent(pair[0]))
        rel, inv = map_rel(pair[1][0])
        rels.append(rel)
        invs.append(inv)
    n_ents = len(ents)
    
    edge_indices = np.concatenate([np.full(n_ents,n_ents)[:,np.newaxis].T, np.arange(0,n_ents)[:,np.newaxis].T], axis=0)
    
    rel_idx_tensor = torch.LongTensor(rels)
    relations = model.relation_embeddings(indices=rel_idx_tensor)
    
    for invix in range(len(invs)):
        if invix == -1:
            relations[invix] = -relations[invix]
    
    ent_idx_tensor = torch.LongTensor(ents)
    source_embeddings = model.entity_embeddings(indices=ent_idx_tensor)
    
    sm = torch.sum(source_embeddings + relations, dim=0)
    return sm

query_name_fn_dict = {'2p': L_p, '3p':L_p, '2i': L_i, '3i':L_i}

In [8]:
allhits1 = []
allhits3 = []
allhits5 = []
allhits10 = []
allmrr = []
query_names = []
target_embeddings = model.entity_embeddings(indices=None)
for query_structure in query_structures:
    print('Running query : {}'.format(query_structure))
    query_name = query_name_dict[query_structure]
    query_names.append(query_name)
    fn = query_name_fn_dict[query_name]
    hits1 = 0.
    hits3 = 0.
    hits5 = 0.
    hits10 = 0.
    mrr = 0.
    cnt = 0
    queries = list(test_queries[query_structure])
    for query in tqdm(queries[:num_test]):
        if len(test_answers[query]) > 0:
            # we have a non-trivial "easy" query
            answers = [map_ent(a) for a in test_answers[query]]
            composite = fn(query, model)
            Q = torch.norm(composite - target_embeddings, dim=1, p=model.scoring_fct_norm).detach().numpy()
            sortd = np.sort(Q)
            idxleft = np.searchsorted(sortd, Q[answers], side='left') + 1
            idxright = np.searchsorted(sortd, Q[answers], side='right') + 1
            # idxright = idxleft # throw this for optimistic ranking
            hits1 += ((np.mean(idxleft <= 1) + np.mean(idxright <= 1)) / 2.)
            hits3 += ((np.mean(idxleft <= 3) + np.mean(idxright <= 3)) / 2.)
            hits5 += ((np.mean(idxleft <= 5) + np.mean(idxright <= 5)) / 2.)
            hits10 += ((np.mean(idxleft <= 10) + np.mean(idxright <= 10)) / 2.)
            mrr += ((np.mean(1./idxleft) + np.mean(1./idxright)) / 2.)
            cnt += 1
    if cnt > 0:
        allhits1.append(hits1/cnt)
        allhits3.append(hits3/cnt)
        allhits5.append(hits5/cnt)
        allhits10.append(hits10/cnt)
        allmrr.append(mrr/cnt)
    else:
        default = 0.
        allhits1.append(default)
        allhits3.append(default)
        allhits5.append(default)
        allhits10.append(default)
        allmrr.append(default)

 15%|█▌        | 30/200 [00:00<00:00, 290.58it/s]

Running query : ('e', ('r', 'r'))


 14%|█▍        | 28/200 [00:00<00:00, 277.48it/s]

Running query : ('e', ('r', 'r', 'r'))


 15%|█▌        | 30/200 [00:00<00:00, 293.66it/s]

Running query : (('e', ('r',)), ('e', ('r',)))


 15%|█▌        | 30/200 [00:00<00:00, 296.95it/s]

Running query : (('e', ('r',)), ('e', ('r',)), ('e', ('r',)))


100%|██████████| 200/200 [00:00<00:00, 291.06it/s]


In [9]:
cols = ['hits@1', 'hits@3', 'hits@5', 'hits@10', 'mrr']
df = pd.DataFrame(np.array([allhits1, allhits3, allhits5, allhits10, allmrr]).T, columns=cols, index=query_names) 

In [10]:
print(model_name)
df * 100 # for percents...

TransE_1000epochs_128embdim_CrossEntropyLossloss_42seed_20210128-1828


,hits@1,hits@3,hits@5,hits@10,mrr
2p,6.539156,19.693430,24.273544,28.713767,14.707361
3p,6.943256,16.726502,19.107179,22.446537,13.119967
2i,3.109791,9.247883,13.803637,19.684356,8.404997
3i,2.489583,9.909295,13.741026,18.606407,8.280399
